In [ ]:
import random
import pandas as pd

In [ ]:
random.seed(42)

In [ ]:
file_path = "../data/tus_new_labeled_union_gt.csv"
df = pd.read_csv(file_path)

In [ ]:
non_union_df = df[df["unionable"] == "no"]
union_df = df[df["unionable"] == "yes"]

In [ ]:
len(non_union_df)

In [ ]:
# Shuffle the rows
shuffled_df = non_union_df.sample(frac=1, random_state=42).reset_index(drop=True)

non_union_filtered_df = pd.DataFrame(data=None, columns=non_union_df.columns)
count_samples_from_each_cluster = {}
for idx, row in shuffled_df.iterrows():
    table_name = row["query_table"]
    data_lake_table_name = row["data_lake_table"]
    column_numbers = int(data_lake_table_name.split("____", 1)[-1].split("_",1)[0][-1])
    cluster_name = str(table_name.split("____",1)[0])
    if column_numbers >= 5:
        # Check if the row exists in the DataFrame
        is_in_df = non_union_filtered_df[(non_union_filtered_df['query_table'] == data_lake_table_name)
                                         & (non_union_filtered_df['data_lake_table'] == table_name)]
        if len(is_in_df) >= 1:
            continue
        if cluster_name not in count_samples_from_each_cluster:
            count_samples_from_each_cluster[cluster_name] = 1
        if count_samples_from_each_cluster[cluster_name] > 20:
            continue
        else:
            count_samples_from_each_cluster[cluster_name] += 1
        non_union_filtered_df = pd.concat([non_union_filtered_df, pd.DataFrame([row])], ignore_index=True)

In [ ]:
len(non_union_filtered_df)

In [ ]:
output_file_path = "../table_union_data/small_tus_non_union_shuffled_sample.csv"
sampled_data = non_union_filtered_df.sample(n=100, random_state=42)
sampled_data.to_csv(output_file_path, index=False)